In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, MaxPool2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train_images = np.load('drive/MyDrive/Colab Notebooks/train_images_kosdaq_change10_2.npy')
# train_labels = np.load('drive/MyDrive/Colab Notebooks/train_labels_kosdaq_change10_2.npy')
test_images = np.load('drive/MyDrive/Colab Notebooks/test_images_kosdaq.npy')
test_labels = np.load('drive/MyDrive/Colab Notebooks/test_labels_kosdaq.npy')

In [ ]:
from sklearn.model_selection import train_test_split

# train_images = np.array(train_images).reshape(-1, 96, 96, 3) / 255.0
test_images = np.array(test_images).reshape(-1, 96, 96, 3) / 255.0

# train_images, test_images, train_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.2)

# train_labels_one_hot = np.eye(2)[train_labels]
test_labels_one_hot = np.eye(2)[test_labels]

In [ ]:
model = Sequential()

# Convolutional Block (Conv-Conv-Pool-Dropout)
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(96, 96, 3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Block (Conv-Conv-Pool-Dropout)
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) # change this line
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
import keras

def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 3:
        lrate = 0.0005
    if epoch > 5:
        lrate = 0.0001
    if epoch > 10:
        lrate = 0.00005

    return lrate

checkpoint = keras.callbacks.ModelCheckpoint('best_cnn_model.h5', save_best_only=True, monitor='val_acc')
# N번의 연속적인 epoch 동안 개선이 없을 때 학습이 중단되도록 함
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=30, restore_best_weights=True)

In [ ]:
from keras.callbacks import LearningRateScheduler

history = model.fit(train_images, train_labels_one_hot, batch_size=32, epochs=100, validation_data=(test_images, test_labels_one_hot),
                    callbacks=[checkpoint, early_stopping, LearningRateScheduler(lr_schedule)])

Epoch 1/100
313/313 [==============================] - 43s 91ms/step - loss: 2.3674 - acc: 0.5079 - val_loss: 0.6932 - val_acc: 0.4980 - lr: 0.0010
Epoch 2/100
313/313 [==============================] - 31s 98ms/step - loss: 0.6910 - acc: 0.5219 - val_loss: 0.6938 - val_acc: 0.5115 - lr: 0.0010
Epoch 3/100
313/313 [==============================] - 26s 83ms/step - loss: 0.6903 - acc: 0.5243 - val_loss: 0.6942 - val_acc: 0.4905 - lr: 0.0010
Epoch 4/100
313/313 [==============================] - 26s 82ms/step - loss: 0.6910 - acc: 0.5241 - val_loss: 0.6919 - val_acc: 0.5030 - lr: 0.0010
Epoch 5/100
313/313 [==============================] - 26s 82ms/step - loss: 0.6888 - acc: 0.5226 - val_loss: 0.6950 - val_acc: 0.5000 - lr: 5.0000e-04
Epoch 6/100
313/313 [==============================] - 25s 81ms/step - loss: 0.6871 - acc: 0.5263 - val_loss: 0.6927 - val_acc: 0.4985 - lr: 5.0000e-04
Epoch 7/100
313/313 [==============================] - 25s 81ms/step - loss: 0.6867 - acc: 0.5204 - val_

In [ ]:
from keras.models import load_model

model40 = load_model('drive/MyDrive/Colab Notebooks/kosdaq_epoch40.h5')
model75 = load_model('drive/MyDrive/Colab Notebooks/kosdaq_epoch75.h5')

In [ ]:
test_loss, test_acc = model40.evaluate(test_images, test_labels_one_hot, verbose=2)
test_loss, test_acc = model75.evaluate(test_images, test_labels_one_hot, verbose=2)

313/313 - 8s - loss: 0.7729 - acc: 0.5024 - 8s/epoch - 27ms/step
313/313 - 6s - loss: 1.1009 - acc: 0.5052 - 6s/epoch - 19ms/step


In [ ]:
# 테스트 데이터셋에 대한 예측 생성
pred_probs40 = model40.predict(test_images)
pred_probs75 = model75.predict(test_images)

313/313 [==============================] - 5s 16ms/step


In [ ]:
# 한 모델만 실험

selected_images = []
selected_labels = []

for i in range(len(test_images)):
    if 0.5 < pred_probs40[i][0] and any(0.792 <= p < 0.82 for p in pred_probs40[i]):
        selected_images.append(test_images[i])
        selected_labels.append(test_labels_one_hot[i])

selected_images = np.array(selected_images)
selected_labels = np.array(selected_labels)

print(len(selected_images))
print(sum(selected_labels))

111
[67. 44.]


In [ ]:
# 두 모델 같은 라벨이 정답이라고 했을때

selected_images = []
selected_labels = []

for i in range(len(test_images)):
    pred_probs75_max_idx = np.argmax(pred_probs75[i])
    pred_probs40_max_idx = np.argmax(pred_probs40[i])

    if pred_probs75_max_idx == pred_probs40_max_idx and pred_probs75_max_idx == 1 and 0.999 <= pred_probs75[i][pred_probs75_max_idx] <= 0.9999 and 0.83 <= pred_probs40[i][pred_probs40_max_idx] <= 0.95:
        selected_images.append(test_images[i])
        selected_labels.append(test_labels_one_hot[i])

selected_images = np.array(selected_images)
selected_labels = np.array(selected_labels)

print(len(selected_images))

26


In [ ]:
# 두 모델 같은 라벨이 정답이라고 했을때

selected_images = []
selected_labels = []

for i in range(len(test_images)):
    pred_probs75_max_idx = np.argmax(pred_probs75[i])
    pred_probs40_max_idx = np.argmax(pred_probs40[i])

    if pred_probs75_max_idx == pred_probs40_max_idx and pred_probs75_max_idx == 0 and 0.97 <= pred_probs75[i][pred_probs75_max_idx] <= 0.99 and 0.79 <= pred_probs40[i][pred_probs40_max_idx] <= 0.86:
        selected_images.append(test_images[i])
        selected_labels.append(test_labels_one_hot[i])

selected_images = np.array(selected_images)
selected_labels = np.array(selected_labels)

print(len(selected_images))

17


In [ ]:
test_loss, test_acc = model75.evaluate(selected_images, selected_labels, verbose=2)
test_loss, test_acc = model40.evaluate(selected_images, selected_labels, verbose=2)

1/1 - 0s - loss: 2.7510 - acc: 0.6538 - 46ms/epoch - 46ms/step
1/1 - 0s - loss: 0.8619 - acc: 0.6538 - 41ms/epoch - 41ms/step


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# 검증 데이터에 대한 예측 값 얻기
y_pred = model.predict(selected_images)

# 예측 결과를 이진 형태로 변환 (임계값을 0.5로 설정)
y_pred_classes = (y_pred > 0.5).astype(int)[:, 0]
test_labels_indices = selected_labels.astype(int)[:, 0]

# Confusion matrix 얻기
tn, fp, fn, tp = confusion_matrix(test_labels_indices, y_pred_classes).ravel()

# 정밀도
precision = tp / (tp + fp)
print("Precision: ", precision)

negative_predictive_alue = tn / (tn + fn)
print("진음성 : ", negative_predictive_alue)

# 재현율 (Recall, Sensitivity, True Positive Rate, TPR) 계산
recall = tp / (tp + fn)
print("Recall: ", recall)

# 특이도 (Specificity, True Negative Rate, TNR) 계산
specificity = tn / (tn + fp)
print("Specificity: ", specificity)

# 위양성율 (False Positive Rate, FPR) 계산
false_positive_rate = fp / (fp + tn)
print("False Positive Rate: ", false_positive_rate)

# 위음성율 (False Negative Rate, FNR) 계산
false_negative_rate = fn / (fn + tp)
print("False Negative Rate: ", false_negative_rate)

f1 = 2 * precision * recall / (precision + recall)
print("F1-score : ", f1)

313/313 [==============================] - 12s 16ms/step
Precision:  0.5050702028081123
진음성 :  0.5053366174055829
Recall:  0.518
Specificity:  0.4924
False Positive Rate:  0.5076
False Negative Rate:  0.482
F1-score :  0.5114533965244866
